In [117]:
import pandas as pd
from os import path
from pathlib import Path

In [118]:
root = Path("./..").resolve()
datasets_folder = root / "exploratory_datasets"
cleaned_data_folder = root / "cleaned_data"
corruption_file = "CPI2022_GlobalResultsTrends.xlsx"


In [119]:
corruption = pd.read_excel(datasets_folder / corruption_file, engine="openpyxl", sheet_name="CPI Timeseries 2012 - 2022", skiprows=2)

In [120]:
corruption_scores = [col_name for col_name in corruption.columns if col_name.startswith("CPI Score") or col_name.startswith("CPI score")]
corruption_sources = [col_name for col_name in corruption.columns if col_name.startswith("Sources")]
standard_errors = [col_name for col_name in corruption.columns if col_name.startswith("Standard error")]
ranks  = [col_name for col_name in corruption.columns if col_name.startswith("Rank")]

In [121]:
remaining_columns = [
    col for col in corruption.columns
    if col not in corruption_sources
       and col not in corruption_scores
       and col not in standard_errors
       and col not in ranks
]

Dropping Rank from data sources as it is not available for all years (only available 2017 - 2022)

In [122]:
corruption = corruption.drop(ranks, axis=1)

Converting from a long to a short column format

In [123]:
corruption_scores_table =  corruption.melt(id_vars = remaining_columns, value_name="corruption_scores", value_vars=corruption_scores, var_name="indicator")
corruption_scores_table['year'] = corruption_scores_table['indicator'].str.split().apply(lambda x: x[-1])
corruption_scores_table = corruption_scores_table.drop("indicator", axis=1)

In [124]:
corruption_sources_table =  corruption.melt(id_vars = remaining_columns, value_name="corruption_sources", value_vars=corruption_sources, var_name="indicator")
corruption_sources_table['year'] = corruption_sources_table['indicator'].str.split().apply(lambda x: x[-1])
corruption_sources_table = corruption_sources_table.drop("indicator", axis=1)

In [125]:
standard_errors_table =  corruption.melt(id_vars = remaining_columns, value_name="corruption_standard_errors", value_vars=corruption_sources, var_name="indicator")
standard_errors_table['year'] = standard_errors_table['indicator'].str.split().apply(lambda x: x[-1])
standard_errors_table = standard_errors_table.drop("indicator", axis=1)

In [126]:
corruption = corruption_sources_table.merge(corruption_scores_table, on=remaining_columns + ["year"]).merge(standard_errors_table, on=remaining_columns + ["year"])

In [131]:
corruption["year"] = corruption['year'].astype(int)

In [132]:
corruption.rename(columns={"Country / Territory": "country_name", "ISO3": "iso_3"}).to_parquet(cleaned_data_folder / 'corruption_index.parquet')